In [87]:
# Dependencies and Setup 
import pandas as pd 
import numpy as np 
 
    
######## BUNU DEĞİŞTİR ##################
######## KAYNAK REHBERI DEĞİŞTİR ########################
##################### File to Load (Remember to Change These) ############################
school_data_to_load = "Resources/schools_complete.csv" 
student_data_to_load = "Resources/students_complete.csv" 
 
# Read School and Student Data File and store into Pandas Data Frames 
school_data = pd.read_csv(school_data_to_load) 
student_data = pd.read_csv(student_data_to_load) 



In [109]:
# FONKSIYONLAR ÖĞRENCİLERIDEN ÖĞRENCİLERIDE (i-onto-i transforms) 

student_data["math_pass"] = np.where(student_data["math_score"]>=60, 1, 0) 
student_data["read_pass"] = np.where(student_data["reading_score"]>=60, 1, 0) 
student_data["grade_pass"] = student_data["math_pass"] * student_data["read_pass"] 


# Evet evet, tabii ki, burada 'binning' kullanmamız gerekiyor. Buyurun işte burda. 
# shkola_df["math_ok"] = pd.cut(shkola_df["math_score"], [0, 60, 100], labels=False, right=False) 
# shkola_df["read_ok"] = pd.cut(shkola_df["reading_score"], [0, 60, 100], labels=False, right=False) 
# shkola_df["grade_ok"] = shkola_df["read_ok"] * shkola_df["math_ok"] 

# student_data.head() 

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,math_pass,read_pass,grade_pass,math_pass_pct,read_pass_pct,grade_pass_pct,avg_pass_pct
0,0,Paul Bradley,M,9th,Huang High School,66,79,1,1,1,100,100,100,100
1,1,Victor Smith,M,12th,Huang High School,94,61,1,1,1,100,100,100,100
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,1,1,1,100,100,100,100
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,1,0,0,100,0,50
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1,1,100,100,100,100


In [89]:
# Compute some school-level values (j-onto-j transform) 
school_data["school_$/stu"] = school_data["budget"] / school_data["size"] 
# school_data.head() 

In [90]:
# Compute some area-level values (j-into-J transforms) 
school_data["school_N"] = school_data["School ID"].count() 
school_data["tot_budget"] = school_data["budget"].sum() 
school_data["tot_$/stu"] = school_data["budget"].sum() / school_data["size"].sum() 

# school_data.head() 

In [91]:
######################## DISTRICT SUMMARY ##################################
######################## DISTRICT SUMMARY ##################################
######################## DISTRICT SUMMARY ##################################
######################## DISTRICT SUMMARY ##################################
            
Xstu_N = student_data["Student ID"].count() 
Xschool_N = school_data["School ID"].count() 
Xbudget_S = school_data["budget"].sum() 
Xmath_MU = student_data["math_score"].mean() 
Xread_MU = student_data["reading_score"].mean() 

Xmath_pass_pct = ((student_data["math_pass"].sum() / student_data["Student ID"].count()) * 100) 
Xread_pass_pct = ((student_data["read_pass"].sum() / student_data["Student ID"].count()) * 100)
Xgrade_pass_pct = ((student_data["grade_pass"].sum() / student_data["Student ID"].count()) * 100)
Xpass_avg_pct = ((Xmath_pass_pct + Xread_pass_pct) / 2) 


#  this is NOT the right way to do formats. 
summary_df = pd.DataFrame({"Total Students": [Xstu_N],
                           "Total Schools": [Xschool_N ],
                           "Total Budget": "$"+str(Xbudget_S),
                           "Avg Math Score": str(round(Xmath_MU))+"%",
                           "Avg Reading Score": str(round(Xread_MU))+"%",
                           "% Pass Math": str(round(Xmath_pass_pct,1))+"%",
                           "% Pass Reading": str(round(Xread_pass_pct,1))+"%",
                           "Avg % Pass": str(round(Xpass_avg_pct,1))+"%",
                           "% Pass Grade": str(round(Xgrade_pass_pct,1))+"%"}) 


# bu. 
summary_df 

,Total Students,Total Schools,Total Budget,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Reading,Avg % Pass,% Pass Grade
0,39170,15,$24649428,79%,82%,92.4%,100.0%,96.2%,92.4%


In [92]:
#SCHOOL DATA PRE-SUMMARY ### 

#  MAKE A NEW DATAFRAME CONTAINING ONLY THE VARIABLES NEEDED, AND CONDITIONED TO HOW I NEED THEM.
student_data["math_pass_pct"] = student_data["math_pass"] * 100
student_data["read_pass_pct"] = student_data["read_pass"] *100 
student_data["grade_pass_pct"] = student_data["grade_pass"] * 100
student_data["avg_pass_pct"] = ((student_data["read_pass"] + student_data["math_pass"]) * 50)

studat_df = student_data.drop([
    'Student ID', 'student_name', 'gender', 'grade', 'math_pass', 'read_pass', 'grade_pass'], 
    axis=1)

# studat_df.head() 

In [93]:
grouped_studat_df = studat_df.groupby(['school_name']) 
# pd.options.display.float_format = '{:.1f}%'.format 
# grouped_studat_df.mean() 

In [94]:
# COMBINE THE DATA INTO ONE BIG FLATFILE DATASET (i,j-into-ij transform)
shkola_df = pd.merge(school_data, grouped_studat_df.mean(), how="left", on=["school_name", "school_name"]) 

# this should be a metric for rankings:  
shkola_df['avg_score'] = ((shkola_df['reading_score'] + shkola_df['math_score']) / 2)

# SO it's going to be one.

# shkola_df 

In [95]:
##################  SCHOOL SUMMARY ######################
##################  SCHOOL SUMMARY ######################
##################  SCHOOL SUMMARY ######################
##################  SCHOOL SUMMARY ######################
##################  SCHOOL SUMMARY ######################
##################  SCHOOL SUMMARY ######################

shkola_df = shkola_df.rename(columns={'school_name': 'School Name', 
                                      'type': 'Type', 'size': '# Students', 
                                      'budget': 'Budget', 
                                      'school_$/stu': 'Budget per Student', 
                                      'reading_score': 'Avg Reading Score', 
                                      'math_score': 'Avg Math Score', 
                                      'avg_score': 'Avg Combined Score',
                                      'read_pass_pct': 'Passing Reading', 
                                      'math_pass_pct': 'Passing Math', 
                                      'grade_pass_pct': 'Passing Grade', 
                                      'avg_pass_pct': 'Overall Passing Rate'}) 

#  "okul" means school. i'm using it as a frame so i can print formatted data but not lose access 
#  to numeric functions on my original dataframe. 
#  i assume that's normal, standard practice. 
#  i know it is.  i didn't know this was a normal, standard 'why' to do that. 
#  yeah, so "okul" means school. 
#  "shkola" also means school. i assume one would guess that. 
#       as does "mektep" if i get desperate -- it shouldn't, it's from an arabic word, in the KTB root. 
#        not sure how it got there, that far north.  
#         maybe they got it from the persians. 
#          one neighborhood, many language groups.  great kitchen, too.  
#            one may acquire a wide range of broad inepitudes in life.  language and cooking are some of mine. 
#             I'm like the Bert Campaneris of culture in a certain part of the world 
#   

okul_df = shkola_df.drop(['School ID', 'school_N', 'tot_budget','tot_$/stu', 'Avg Combined Score'], axis=1)

format_mapping={'Budget': '${:,.0f}', 
                'Budget per Student': '${:,.2f}', 
                'Avg Reading Score': '{:.1f}%', 
                'Avg Math Score': '{:.1f}%', 
                'Passing Math': '{:.1f}%', 
                'Passing Reading': '{:.1f}%', 
                'Passing Grade': '{:.1f}%', 
                'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    okul_df[key] = okul_df[key].apply(value.format) 

#  NB: Overall passing rate may be a misnomer. 

okul_df 

,School Name,Type,# Students,Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate
0,Huang High School,District,2917,"$1,910,635",$655.00,81.2%,76.6%,88.9%,100.0%,88.9%,94.4%
1,Figueroa High School,District,2949,"$1,884,411",$639.00,81.2%,76.7%,88.4%,100.0%,88.4%,94.2%
2,Shelton High School,Charter,1761,"$1,056,600",$600.00,83.7%,83.4%,100.0%,100.0%,100.0%,100.0%
3,Hernandez High School,District,4635,"$3,022,020",$652.00,80.9%,77.3%,89.1%,100.0%,89.1%,94.5%
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%
5,Wilson High School,Charter,2283,"$1,319,574",$578.00,84.0%,83.3%,100.0%,100.0%,100.0%,100.0%
6,Cabrera High School,Charter,1858,"$1,081,356",$582.00,84.0%,83.1%,100.0%,100.0%,100.0%,100.0%
7,Bailey High School,District,4976,"$3,124,928",$628.00,81.0%,77.0%,89.5%,100.0%,89.5%,94.8%
8,Holden High School,Charter,427,"$248,087",$581.00,83.8%,83.8%,100.0%,100.0%,100.0%,100.0%
9,Pena High School,Charter,962,"$585,858",$609.00,84.0%,83.8%,100.0%,100.0%,100.0%,100.0%


In [97]:
####### TOP EIGHT PERFORMING SCHOOLS #### 
####### TOP EIGHT PERFORMING SCHOOLS #### 
####### TOP EIGHT PERFORMING SCHOOLS #### 
####### TOP EIGHT PERFORMING SCHOOLS #### 
####### TOP EIGHT PERFORMING SCHOOLS #### 

# -- NON-UNIQUE BECAUSE EIGHT OF THEM TIE.

okul_df = shkola_df.drop(['School ID', 'school_N', 'tot_budget','tot_$/stu', 'Avg Combined Score'], axis=1)

format_mapping={'Budget': '${:,.0f}', 
                'Budget per Student': '${:,.2f}', 
                'Avg Reading Score': '{:.1f}%', 
                'Avg Math Score': '{:.1f}%', 
                'Passing Math': '{:.1f}%', 
                'Passing Reading': '{:.1f}%', 
                'Passing Grade': '{:.1f}%', 
                'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    okul_df[key] = okul_df[key].apply(value.format) 
    

okul_df = okul_df.sort_values('Overall Passing Rate') 
okul_df["rank1"] = okul_df["Overall Passing Rate"].rank() 

okul_df = okul_df.sort_values('rank1') 

top8_df = okul_df[okul_df['rank1'] <= 5] 

# since everybody left has the same rank, and for appearance, drop these from top8_df
top8_df = top8_df.drop(['rank1'], axis=1)

top8_df 

,School Name,Type,# Students,Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate
2,Shelton High School,Charter,1761,"$1,056,600",$600.00,83.7%,83.4%,100.0%,100.0%,100.0%,100.0%
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%
5,Wilson High School,Charter,2283,"$1,319,574",$578.00,84.0%,83.3%,100.0%,100.0%,100.0%,100.0%
6,Cabrera High School,Charter,1858,"$1,081,356",$582.00,84.0%,83.1%,100.0%,100.0%,100.0%,100.0%
8,Holden High School,Charter,427,"$248,087",$581.00,83.8%,83.8%,100.0%,100.0%,100.0%,100.0%
9,Pena High School,Charter,962,"$585,858",$609.00,84.0%,83.8%,100.0%,100.0%,100.0%,100.0%
10,Wright High School,Charter,1800,"$1,049,400",$583.00,84.0%,83.7%,100.0%,100.0%,100.0%,100.0%
14,Thomas High School,Charter,1635,"$1,043,130",$638.00,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%


In [99]:
######  TOP SIX  ######  TOP SIX  ######  TOP SIX  ###### 
######  TOP SIX  ######  TOP SIX  ######  TOP SIX  ###### 
### FORCE FIT SO WE GET A SET OF FIVE . . . OR SIX. 
# The goal was to get the top five. No can do. So here are the top six. 

okul_df = shkola_df.drop(['School ID', 'school_N', 'tot_budget','tot_$/stu'], axis=1)

format_mapping={'Budget': '${:,.0f}', 
                'Budget per Student': '${:,.2f}', 
                'Avg Reading Score': '{:.1f}%', 
                'Avg Math Score': '{:.1f}%', 
                'Avg Combined Score': '{:.1f}%', 
                'Passing Math': '{:.1f}%', 
                'Passing Reading': '{:.1f}%', 
                'Passing Grade': '{:.1f}%', 
                'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    okul_df[key] = okul_df[key].apply(value.format) 
    
okul_df = okul_df.sort_values('Avg Combined Score', ascending=False) 
okul_df["rank"] = okul_df["Avg Combined Score"].rank(ascending=False) 

top6_df = okul_df[okul_df['rank'] <= 5] 

top6_df 

,School Name,Type,# Students,Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate,Avg Combined Score,rank
9,Pena High School,Charter,962,"$585,858",$609.00,84.0%,83.8%,100.0%,100.0%,100.0%,100.0%,83.9%,1.0
8,Holden High School,Charter,427,"$248,087",$581.00,83.8%,83.8%,100.0%,100.0%,100.0%,100.0%,83.8%,2.5
10,Wright High School,Charter,1800,"$1,049,400",$583.00,84.0%,83.7%,100.0%,100.0%,100.0%,100.0%,83.8%,2.5
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%,83.6%,5.0
5,Wilson High School,Charter,2283,"$1,319,574",$578.00,84.0%,83.3%,100.0%,100.0%,100.0%,100.0%,83.6%,5.0
14,Thomas High School,Charter,1635,"$1,043,130",$638.00,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%,83.6%,5.0


In [104]:
##### BOTTTOM FIVE SCHOOLS #####    ##### BOTTOM FIBE SCHOOLS #####    ##### BLOTTO FIIINE SCHOOSH  ##### 
##### BOTTOOM FIVE SCHOOL #####    ##### BOTTON FINE SCHOOLS ##### 
##### BOTTTOM FIEV SCHOLS #####    ##### BOTTOM FIBE SHOOLS #####    ##### BLOTTO FIIINE SCHOOSH  ##### 
##### BOTTOOM FIVE SCHOOLS #####    ##### BOTTON FINE SCHOLS ##### 
##### BOTTTOM VIFE SCHOOLS #####    ##### BOTTOM FIBE SCOOLS #####    ##### BLOTTO FIIINE SCHOOSH  ##### 
##### BOTTOOM FIVE SCHOOLS #####    ##### BOTTON FIRE SCHOOLS ##### 

okul_df = shkola_df.drop(['School ID', 'school_N', 'tot_budget','tot_$/stu'], axis=1)

format_mapping={'Budget': '${:,.0f}', 
                'Budget per Student': '${:,.2f}', 
                'Avg Reading Score': '{:.1f}%', 
                'Avg Math Score': '{:.1f}%', 
                'Avg Combined Score': '{:.1f}%', 
                'Passing Math': '{:.1f}%', 
                'Passing Reading': '{:.1f}%', 
                'Passing Grade': '{:.1f}%', 
                'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    okul_df[key] = okul_df[key].apply(value.format) 

okul_df = okul_df.sort_values('Overall Passing Rate', ascending=False) 
okul_df["Worst"] = okul_df["Overall Passing Rate"].rank(ascending=False) 

bottom5_df = okul_df[okul_df['Worst'] <= 5] 
bottom5_df = bottom5_df.sort_values('Worst') 

bottom5_df 

,School Name,Type,# Students,Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate,Avg Combined Score,Worst
7,Bailey High School,District,4976,"$3,124,928",$628.00,81.0%,77.0%,89.5%,100.0%,89.5%,94.8%,79.0%,1.0
13,Ford High School,District,2739,"$1,763,916",$644.00,80.7%,77.1%,89.3%,100.0%,89.3%,94.7%,78.9%,2.0
12,Johnson High School,District,4761,"$3,094,650",$650.00,81.0%,77.1%,89.2%,100.0%,89.2%,94.6%,79.0%,3.0
3,Hernandez High School,District,4635,"$3,022,020",$652.00,80.9%,77.3%,89.1%,100.0%,89.1%,94.5%,79.1%,4.0
0,Huang High School,District,2917,"$1,910,635",$655.00,81.2%,76.6%,88.9%,100.0%,88.9%,94.4%,78.9%,5.0


In [108]:
### MATH SCORES BY GRADE AND SCHOOLS ####### 
### MATH SCORES BY GRADE AND SCHOOLS ####### 
### MATH SCORES BY GRADE AND SCHOOLS ####### 
### MATH SCORES BY GRADE AND SCHOOLS ####### 


# I CAN'T GET THIS INTO AN X-Y CROSSTAB WHERE SCHOOL IS X AXIS AND GRADE IS Y AXIS AND DATA IS FILL. 

stumath_df = student_data.drop(['reading_score', 
                                'Student ID', 'student_name', 'gender', 
                                'math_pass', 'read_pass', 'grade_pass', 
                                'math_pass_pct', 'read_pass_pct', 'grade_pass_pct', 'avg_pass_pct'], axis=1) 

stumath_df = stumath_df.rename(columns={'school_name': 'School Name', 
                                        'grade': "Grade", 
                                        'math_score': 'Avg Math Score'})


stumath_df = stumath_df.groupby(['School Name', 'Grade']).mean()  

stumath_df.pivot_table('Avg Math Score', 'School Name', 'Grade', fill_value=0)

format_mapping = {'Avg Math Score': '{:.1f}%'}

for key, value in format_mapping.items():
    stumath_df[key] = stumath_df[key].apply(value.format) 
    
stumath_df


Avg Math Score
School Name           Grade               
Bailey High School    10th           77.0%
                      11th           77.5%
                      12th           76.5%
                      9th            77.1%
Cabrera High School   10th           83.2%
                      11th           82.8%
                      12th           83.3%
                      9th            83.1%
Figueroa High School  10th           76.5%
                      11th           76.9%
                      12th           77.2%
                      9th            76.4%
Ford High School      10th           77.7%
                      11th           76.9%
                      12th           76.2%
                      9th            77.4%
Griffin High School   10th           84.2%
                      11th           83.8%
                      12th           83.4%
                      9th            82.0%
Hernandez High School 10th           77.3%
                      11th           77.1%
                      12th           77.2%
                      9th            77.4%
Holden High School    10th           83.4%
                      11th           85.0%
                      12th           82.9%
                      9th            83.8%
Huang High School     10th           75.9%
                      11th           76.4%
                      12th           77.2%
                      9th            77.0%
Johnson High School   10th           76.7%
                      11th           77.5%
                      12th           76.9%
                      9th            77.2%
Pena High School      10th           83.4%
                      11th           84.3%
                      12th           84.1%
                      9th            83.6%
Rodriguez High School 10th           76.6%
                      11th           76.4%
                      12th           77.7%
                      9th            76.9%
Shelton High School   10th           82.9%
                      11th           83.4%
                      12th           83.8%
                      9th            83.4%
Thomas High School    10th           83.1%
                      11th           83.5%
                      12th           83.5%
                      9th            83.6%
Wilson High School    10th           83.7%
                      11th           83.2%
                      12th           83.0%
                      9th            83.1%
Wright High School    10th           84.0%
                      11th           83.8%
                      12th           83.6%
                      9th            83.3%

In [111]:
###################

### READING SCORES BY GRADE AND SCHOOLS ####### 
### READING SCORES BY GRADE AND SCHOOLS ####### 
### READING SCORES BY GRADE AND SCHOOLS ####### 
### READING SCORES BY GRADE AND SCHOOLS ####### 


sturead_df = student_data.drop(['math_score', 
                                'Student ID', 'student_name', 'gender', 
                                'math_pass', 'read_pass', 'grade_pass', 
                                'math_pass_pct', 'read_pass_pct', 'grade_pass_pct', 'avg_pass_pct'], axis=1) 

sturead_df = sturead_df.rename(columns={'school_name': 'School Name', 
                                        'grade': "Grade", 
                                        'reading_score': 'Avg Reading Score'})

#### i didn't realize until later that i could do this:
####  << df = df[[var1, var3, var99]]  >>
###  and get a reduced dataset.  


sturead_df = sturead_df.groupby(['School Name', 'Grade']).mean()  



sturead_df.pivot_table('Avg Reading Score', 'School Name', 'Grade', fill_value=0)

#  seriously, try to put the format above the pivot or groupby statements.  
#  it  bugs.

format_mapping = {'Avg Reading Score': '{:.1f}%'}

for key, value in format_mapping.items():
    sturead_df[key] = sturead_df[key].apply(value.format) 
    
sturead_df



Avg Reading Score
School Name           Grade                  
Bailey High School    10th              80.9%
                      11th              80.9%
                      12th              80.9%
                      9th               81.3%
Cabrera High School   10th              84.3%
                      11th              83.8%
                      12th              84.3%
                      9th               83.7%
Figueroa High School  10th              81.4%
                      11th              80.6%
                      12th              81.4%
                      9th               81.2%
Ford High School      10th              81.3%
                      11th              80.4%
                      12th              80.7%
                      9th               80.6%
Griffin High School   10th              83.7%
                      11th              84.3%
                      12th              84.0%
                      9th               83.4%
Hernandez High School 10th              80.7%
                      11th              81.4%
                      12th              80.9%
                      9th               80.9%
Holden High School    10th              83.3%
                      11th              83.8%
                      12th              84.7%
                      9th               83.7%
Huang High School     10th              81.5%
                      11th              81.4%
                      12th              80.3%
                      9th               81.3%
Johnson High School   10th              80.8%
                      11th              80.6%
                      12th              81.2%
                      9th               81.3%
Pena High School      10th              83.6%
                      11th              84.3%
                      12th              84.6%
                      9th               83.8%
Rodriguez High School 10th              80.6%
                      11th              80.9%
                      12th              80.4%
                      9th               81.0%
Shelton High School   10th              83.4%
                      11th              84.4%
                      12th              82.8%
                      9th               84.1%
Thomas High School    10th              84.3%
                      11th              83.6%
                      12th              83.8%
                      9th               83.7%
Wilson High School    10th              84.0%
                      11th              83.8%
                      12th              84.3%
                      9th               83.9%
Wright High School    10th              83.8%
                      11th              84.2%
                      12th              84.1%
                      9th               83.8%

In [148]:
# shkola_df 

In [132]:
#####  SCORES BY SCHOOL SPENDING ####
#####  SCORES BY SCHOOL SPENDING ####
#####  SCORES BY SCHOOL SPENDING ####
    

moneycat_df = shkola_df.sort_values('Budget per Student') 

bins = [575, 599, 630, 649, 675]
group_names = ["Low", "Average", "Above Average", "High"]

moneycat_df["Spend Cat"] = pd.cut(moneycat_df['Budget per Student'], bins, labels=group_names)

moneycat_df = moneycat_df.drop(['School ID', 'Type', '# Students', 'Budget', 'school_N', 'tot_budget', 'tot_$/stu'], axis = 1)

moneycat_df = moneycat_df[['School Name', 'Budget per Student', 
                          'Avg Reading Score', 'Avg Math Score', 'Avg Combined Score', 
                          'Passing Math', 'Passing Reading', 'Passing Grade', 'Overall Passing Rate']]

format_mapping = {'Budget per Student': '${:,.2f}', 
                  'Avg Reading Score': '{:.1f}%', 
                  'Avg Math Score': '{:.1f}%', 
                  'Avg Combined Score': '{:.1f}%', 
                  'Passing Math': '{:.1f}%', 
                  'Passing Reading': '{:.1f}%', 
                  'Passing Grade': '{:.1f}%', 
                  'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    moneycat_df[key] = moneycat_df[key].apply(value.format) 

moneycat_df


,School Name,Budget per Student,Avg Reading Score,Avg Math Score,Avg Combined Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate
5,Wilson High School,$578.00,84.0%,83.3%,83.6%,100.0%,100.0%,100.0%,100.0%
8,Holden High School,$581.00,83.8%,83.8%,83.8%,100.0%,100.0%,100.0%,100.0%
6,Cabrera High School,$582.00,84.0%,83.1%,83.5%,100.0%,100.0%,100.0%,100.0%
10,Wright High School,$583.00,84.0%,83.7%,83.8%,100.0%,100.0%,100.0%,100.0%
2,Shelton High School,$600.00,83.7%,83.4%,83.5%,100.0%,100.0%,100.0%,100.0%
9,Pena High School,$609.00,84.0%,83.8%,83.9%,100.0%,100.0%,100.0%,100.0%
4,Griffin High School,$625.00,83.8%,83.4%,83.6%,100.0%,100.0%,100.0%,100.0%
7,Bailey High School,$628.00,81.0%,77.0%,79.0%,89.5%,100.0%,89.5%,94.8%
11,Rodriguez High School,$637.00,80.7%,76.8%,78.8%,88.5%,100.0%,88.5%,94.3%
14,Thomas High School,$638.00,83.8%,83.4%,83.6%,100.0%,100.0%,100.0%,100.0%


In [135]:
##############  SIZECAT ###################
##############  SIZECAT ###################
##############  SIZECAT ###################

##############  SIZECAT ###################

sizecat_df = shkola_df.sort_values('# Students') 

bins = [400, 1000, 2000, 3000, 5000] 
group_names = ["Small", "Normal", "Large", "Giant"] 

sizecat_df["Size Cat"] = pd.cut(sizecat_df['# Students'], bins, labels=group_names) 

sizecat_df = sizecat_df[['School Name', '# Students', 'Size Cat', 'Avg Reading Score', 'Avg Math Score', 'Avg Combined Score', 
                          'Passing Math', 'Passing Reading', 'Passing Grade', 'Overall Passing Rate']] 

format_mapping = {'Avg Reading Score': '{:.1f}%', 
                  'Avg Math Score': '{:.1f}%', 
                  'Avg Combined Score': '{:.1f}%', 
                  'Passing Math': '{:.1f}%', 
                  'Passing Reading': '{:.1f}%', 
                  'Passing Grade': '{:.1f}%', 
                  'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    sizecat_df[key] = sizecat_df[key].apply(value.format) 

    
sizecat_df

,School Name,# Students,Size Cat,Avg Reading Score,Avg Math Score,Avg Combined Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate
8,Holden High School,427,Small,83.8%,83.8%,83.8%,100.0%,100.0%,100.0%,100.0%
9,Pena High School,962,Small,84.0%,83.8%,83.9%,100.0%,100.0%,100.0%,100.0%
4,Griffin High School,1468,Normal,83.8%,83.4%,83.6%,100.0%,100.0%,100.0%,100.0%
14,Thomas High School,1635,Normal,83.8%,83.4%,83.6%,100.0%,100.0%,100.0%,100.0%
2,Shelton High School,1761,Normal,83.7%,83.4%,83.5%,100.0%,100.0%,100.0%,100.0%
10,Wright High School,1800,Normal,84.0%,83.7%,83.8%,100.0%,100.0%,100.0%,100.0%
6,Cabrera High School,1858,Normal,84.0%,83.1%,83.5%,100.0%,100.0%,100.0%,100.0%
5,Wilson High School,2283,Large,84.0%,83.3%,83.6%,100.0%,100.0%,100.0%,100.0%
13,Ford High School,2739,Large,80.7%,77.1%,78.9%,89.3%,100.0%,89.3%,94.7%
0,Huang High School,2917,Large,81.2%,76.6%,78.9%,88.9%,100.0%,88.9%,94.4%


In [147]:
######## SCHOOL TYPE ############
######## SCHOOL TYPE ############
######## SCHOOL TYPE #################### SCHOOL TYPE ############

typecat_df = shkola_df.sort_values('Type') 


typecat_df = typecat_df[['School Name', '# Students', 'Avg Reading Score', 'Avg Math Score',
                         'Passing Math', 'Passing Reading', 'Passing Grade', 'Overall Passing Rate', 'Type']] 

format_mapping = {'Avg Reading Score': '{:.1f}%', 
                  'Avg Math Score': '{:.1f}%', 
                  'Passing Math': '{:.1f}%', 
                  'Passing Reading': '{:.1f}%', 
                  'Passing Grade': '{:.1f}%', 
                  'Overall Passing Rate': '{:.1f}%'} 

for key, value in format_mapping.items(): 
    typecat_df[key] = typecat_df[key].apply(value.format) 

    
typecat_df

,School Name,# Students,Avg Reading Score,Avg Math Score,Passing Math,Passing Reading,Passing Grade,Overall Passing Rate,Type
8,Holden High School,427,83.8%,83.8%,100.0%,100.0%,100.0%,100.0%,Charter
9,Pena High School,962,84.0%,83.8%,100.0%,100.0%,100.0%,100.0%,Charter
4,Griffin High School,1468,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%,Charter
14,Thomas High School,1635,83.8%,83.4%,100.0%,100.0%,100.0%,100.0%,Charter
2,Shelton High School,1761,83.7%,83.4%,100.0%,100.0%,100.0%,100.0%,Charter
10,Wright High School,1800,84.0%,83.7%,100.0%,100.0%,100.0%,100.0%,Charter
6,Cabrera High School,1858,84.0%,83.1%,100.0%,100.0%,100.0%,100.0%,Charter
5,Wilson High School,2283,84.0%,83.3%,100.0%,100.0%,100.0%,100.0%,Charter
13,Ford High School,2739,80.7%,77.1%,89.3%,100.0%,89.3%,94.7%,District
0,Huang High School,2917,81.2%,76.6%,88.9%,100.0%,88.9%,94.4%,District
